In [1]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

### 3.7.1. 初始化模型参数

In [2]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

### 3.7.2. 重新审视Softmax的实现

In [3]:
loss = nn.CrossEntropyLoss(reduction='none')

### 3.7.3. 优化算法

In [4]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

### 3.7.4. 训练

In [5]:
num_epochs = 10
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

### Practice

#### 1. 尝试调整超参数，例如批量大小、迭代周期数和学习率，并查看结果。

**解答：**

调整超参数的影响如下：
- **批量大小**：
  - 小批量大小：模型更新频率高，能更快收敛，但可能导致更新噪声较大，模型容易陷入局部最优。
  - 大批量大小：更新更稳定，但训练效率可能较低，且容易导致欠拟合。

- **迭代周期数**：
  - 较少的迭代周期：可能导致模型未充分训练，精度不足。
  - 较多的迭代周期：可能导致过拟合，模型在训练集上的表现很好，但在测试集上效果变差。

- **学习率**：
  - 学习率过小：收敛速度慢，可能需要较长时间才能达到最优解。
  - 学习率过大：可能导致模型不稳定，甚至无法收敛。

优化策略为：
- 使用验证集观察模型性能，选择合适的超参数。
- 初始时采用较大的学习率，并在训练过程中逐步衰减（如使用学习率调度器）。

---

#### 2. 增加迭代周期的数量。为什么测试精度会在一段时间后降低？我们怎么解决这个问题？

**解答：**

- **原因：**
  测试精度在一段时间后降低是由于**过拟合**问题。随着迭代周期的增加，模型在训练集上的表现越来越好，但可能会开始过度拟合训练数据中的噪声，从而导致在测试集上的泛化性能下降。

- **解决方法：**
  1. **早停法（Early Stopping）**：
     在训练过程中，持续监控验证集的性能，当验证集的损失在若干迭代后不再下降时停止训练。

  2. **正则化**：
     - 添加正则化项（如 $L_1$ 或 $L_2$ 正则化）以限制模型的复杂度。
     - 增加Dropout层，随机丢弃部分神经元，减少过拟合。

  3. **数据增强**：
     增加训练数据的多样性（如图像旋转、翻转、缩放等），提高模型的泛化能力。

  4. **减少模型复杂度**：
     使用更小的网络或减少参数数量，降低模型过拟合的风险。

  5. **学习率调整**：
     在后期逐步降低学习率，使模型在接近最优解时能够更稳定地收敛。

---